In [ ]:
# Importando bibliotecas
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
# Carregamento dos Dados
df = pd.read_excel("conjunto-de-dados.xlsx")

# Separação de descritores (X) e variável alvo (y)
X = df.drop(columns=["variavel-alvo"])
y = df["variavel-alvo"]

# Divisão em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Normalização dos Dados
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
# Avaliação de hiperparâmetros
lgbm = lgb.LGBMRegressor(random_state=42, force_row_wise=True)

param_grid_lgbm = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_samples': [5, 10, 20],
    'min_split_gain': [0.0, 0.1, 0.2]
}

grid_lgbm = GridSearchCV(estimator=lgbm, param_grid=param_grid_lgbm, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_lgbm.fit(X_train_scaled, y_train)

best_lgbm = grid_lgbm.best_estimator_
# Modelagem
y_train_pred_lgbm = best_lgbm.predict(X_train_scaled)
y_pred_lgbm = best_lgbm.predict(X_test_scaled)

mse_train_lgbm = mean_squared_error(y_train, y_train_pred_lgbm)
r2_train_lgbm = r2_score(y_train, y_train_pred_lgbm)
mae_train_lgbm = mean_absolute_error(y_train, y_train_pred_lgbm)

mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
r2_lgbm = r2_score(y_test, y_pred_lgbm)
mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)

print(f"LightGBM - Métricas de Treinamento:")
print(f"  - Erro quadrático médio (MSE) no treino: {mse_train_lgbm:.4f}")
print(f"  - R² no treino: {r2_train_lgbm:.4f}")
print(f"  - Erro absoluto médio (MAE) no treino: {mae_train_lgbm:.4f}")

print(f"LightGBM - Métricas de Teste (Validação):")
print(f"  - Erro quadrático médio (MSE) no teste: {mse_lgbm:.4f}")
print(f"  - R² no teste: {r2_lgbm:.4f}")
print(f"  - Erro absoluto médio (MAE) no teste: {mae_lgbm:.4f}")
print(f"Melhores parâmetros para LightGBM: {grid_lgbm.best_params_}")
# Visualização dos Resultados (Previsões vs Valores Reais)
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_lgbm, color='blue', label='Previsões')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Linha de identidade')
plt.xlabel('Valores Reais')
plt.ylabel('Previsões')
plt.title('Previsões vs. Valores Reais - Light GBM')
plt.legend()
plt.show()